# Tutorial notebook: [minimal working example](https://firefly.rcs.northwestern.edu/docs/data_reader/minimal_example.html?highlight=minimal)
Here we use some minimal example data.

In [1]:
import numpy as np

In [2]:
# Load your data. Here we're creating random data.
coords = np.random.randn( 20000, 3 )
fields = np.random.random(size=coords[:,0].size)

## Format Data for Firefly

In [3]:
from firefly.data_reader import ArrayReader

Here are the [docs for ArrayReader](https://firefly.rcs.northwestern.edu/docs/reference/api/readers/firefly.data_reader.ArrayReader.html).

In [ ]:
my_arrayReader = ArrayReader(
    coords,
    fields=fields,
    write_to_disk=False)

## Display Inline

In [5]:
from firefly.server import spawnFireflyServer,quitAllFireflyServers

In [6]:
# define the local port (typically anything in 5000 - 8000 range)
port = 5500

In [ ]:
process = spawnFireflyServer(port, method = 'flask')

In [ ]:
from IPython.display import IFrame
url = f'http://localhost:{port:d}/combined'
IFrame(url, width=1000, height=500)

In [ ]:
# Send data to the server.
# Wait until it loads to run this command
my_arrayReader.sendDataViaFlask()

In [ ]:
return_code = quitAllFireflyServers()